In [99]:
import numpy as np 
import pandas as pd 
import random

In [100]:
class colors():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['S','H','D','C'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [101]:
class numbers():
    
    __slots__ = '__possibilities'
    
    def __init__(self):
        self.__possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A'] #no need to ensure type
    
    def getPossibilities(self):
        return self.__possibilities

In [115]:
class card():
    __slots__ = '__color', '__number'
    
    def __init__(self,aColor, aNumber):
        
        self.__color = aColor
        self.__number = aNumber
        
    def getColor():
        return self.__color
    
    def getNumber():
        return self.__number
    

In [103]:
class hand():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard):
        self.__cards = [self.__cards, aCard]
        self.__numbersOnly = [self.__numbersUnique, aCard.getNumber()]
        self.__colorsOnly = [self.__colorsOnly, aCard.getColor()]
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly

In [137]:
class deck():
    __slots__ = '__cards'
        
    def __init__(self):
        self.__cards = []
        for color in colors().getPossibilities():
            for number in numbers().getPossibilities():
                self.__cards = [self.__cards, card(color, number)]
        
    
    def drawCard(self):
        return self.__cards.pop(0)
    
    def getDeck(self):
        return self.__cards

In [105]:
class river():
    __slots__ = '__cards', '__size', '__numbersOnly', '__colorsOnly'
    
    def __init__(self):
        self.__size = 0
        self.__cards = []
        self.__numbersOnly = []
        self.__colorsOnly = []
        
    def addCard(self, aCard):
        self.__cards = [self.__cards, aCard]
        self.__numbersOnly = [self.__numbersUnique, aCard.getNumber()]
        self.__colorsOnly = [self.__colorsOnly, aCard.getColor()]
        self.__size = self.__size + 1
        
    def getCards(self):
        return self.__cards
    
    def getNumbers(self):
        return self.__numbersOnly
    
    def getColors(self):
        return self.__colorsOnly

In [106]:
def isFlush(aHand):
    
    #init
    sameColorCount = 1
    first = True
    
    pastColor = ''
    for card in aHand.getCards():
        if not first:
            if card.getColor() == pastColor:
                sameColorCount = sameColorCount+1
            
        pastColor = card.getColor()
        first = False
        
    return sameColorCount == 5

def isStraight(aHand):
    myHand = sorted(np.unique(aHand.getNumbers), key=lambda x: numbers().getPossibilities().index(x))

def howManyPairs(aHand):
    
def singleCard():

IndentationError: expected an indented block (<ipython-input-106-f23bddce383b>, line 23)

In [107]:
def getValueHand(aHand):
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-107-53df8bed29b8>, line 3)

In [108]:
def getWinner(river, hands):
    #return index of winning hands
    values = []
    
    for hand in hands:
        myHandToEval = [hand.getCards(), river.getCards()]
        values = [values, getValueHand(hand)]
        
        
    
    

In [109]:
a = ['K', 'J', 'A','A'] 
possibilities = ['2','3','4','5','6','7','8','9','J','Q','K','A']

sorted(np.unique(a), key=lambda x: possibilities.index(x))

['J', 'K', 'A']

In [138]:
myDeck = deck()
myHands = []
for i in range(0,2):
    myHands.append(hand())
myRiver = river()

In [139]:
#draw cards
for i in range(0,3):
    for j in range(0,2):
        myHands[j].addCard(myDeck.drawCard())

AttributeError: 'hand' object has no attribute '_hand__numbersUnique'